# Tools

> Helper functions for polcies APIs processing

In [ ]:
#| default_exp policy/helper/tools

In [ ]:
# | export
def apply_markdown(
    doamin,  # Single `Domain` object of standard
    index,  # Index of domain in the standard
) -> str:  # Markdown string of the entire domain
    """
    Generate controls Markdown for generate policies and controls, domain should contain(domainId, Control[controlId, name and description])

    - Apply markdown on control `name` & `description`
    - Apply Indexing on `DomainId`
    - Apply Indexing on `controlId`
    """
    doamin["DomainId"] = f'{index}:{doamin["DomainId"]}'
    markdown_output = []
    for control in doamin["Controls"]:
        control["controlId"] = doamin["DomainId"] + ":" + control["controlId"]
        mkd = (
            f"**{control['controlId']} - {control['name']}**: {control['description']}"
        )
        markdown_output.append(mkd)
        control["markdown"] = mkd
    return markdown_output

In [ ]:
domain = {
    "DomainId": "4",
    "Domain": "Performance evaluation",
    "Controls": [
        {
            "controlId": "4.1",
            "name": "Monitoring",
            "description": "Description of the Monitoring control",
        },
        {
            "controlId": "4.1.1",
            "name": "Measurement",
            "description": "Description of the Measurement control",
        },
    ],
}
apply_markdown(domain, 0)

['**0:4:4.1 - Monitoring**: Description of the Monitoring control',
 '**0:4:4.1.1 - Measurement**: Description of the Measurement control']

In [ ]:
#| hide
# Test case for apply_markdown
domain = {
    "DomainId": "4",
    "Domain": "Performance evaluation",
    "Controls": [
        {
            "controlId": "4.1",
            "name": "Monitoring",
            "description": "Description of the Monitoring control",
        },
        {
            "controlId": "4.1.1",
            "name": "Measurement",
            "description": "Description of the Measurement control",
        },
    ],
}
expected_output = [
    "**0:4:4.1 - Monitoring**: Description of the Monitoring control",
    "**0:4:4.1.1 - Measurement**: Description of the Measurement control"
]
def_output = apply_markdown(domain, 0)
assert expected_output == def_output # success test case

# Failed test case
domain = {
    "DomainId": "4",
    "Domain": "Performance evaluation",
    "Controls": [
        {
            "controlId": "4.1",
            "name": "Monitoring",
            "description": "Description of the Monitoring control",
        }
    ],
}
expected_output_fail = [
    "**1:4:4.1 - Monitoring**: Description of the Monitoring control",
]
def_output = apply_markdown(domain, 0)
assert not expected_output_fail == def_output

In [ ]:
# | export
def apply_markdown_gap(
    doamin, # Single Domain object of standard 
    index # Index of domain in the standard
) -> str:  # Markdown string of the entire domain
    """
    Generate controls Markdown for gap analysis, domain should contain(domainId, Control[controlId, name and description])

    - Apply markdown on control `name` & `description`
    - Apply Indexing on `DomainId`
    - Apply Indexing on `controlId`
    """
    doamin["domain_number"] = f'{index}:{doamin["domain_number"]}'
    markdown_output = []
    for control in doamin["controls"]:
        control["control_number"] = doamin["domain_number"] + ":" + control["control_number"]
        mkd = f"**{control['control_number']} - {control['control_name']}**: {control['control_description']}"
        markdown_output.append(mkd)
        control["markdown"] = mkd
    return markdown_output

In [ ]:
domain = {
    "domain_name": "Access Control",
    "domain_number": "8",
    "controls": [
        {
            "control_number": "8.1",
            "control_name": "User Endpoint Devices",
            "control_description": "Ensure that information stored on user devices is protected with security configurations and policies."
        },
        {
            "control_number": "8.3",
            "control_name": "Remote Access Control",
            "control_description": "Ensure secure remote access to information systems and networks."
        }
    ]
}
apply_markdown_gap({**domain},0)

['**0:8:8.1 - User Endpoint Devices**: Ensure that information stored on user devices is protected with security configurations and policies.',
 '**0:8:8.3 - Remote Access Control**: Ensure secure remote access to information systems and networks.']

In [ ]:
#| hide #Test case


domain = {
    "domain_name": "Access Control",
    "domain_number": "8",
    "controls": [
        {
            "control_number": "8.1",
            "control_name": "User Endpoint Devices",
            "control_description": "Ensure that information stored on user devices is protected with security configurations and policies."
        }
    ]
}
expected_output = [
    "**0:8:8.1 - User Endpoint Devices**: Ensure that information stored on user devices is protected with security configurations and policies."
]
def_output = apply_markdown_gap(domain, 0)
assert expected_output == def_output #success test case
# Failed test case
domain = {
    "domain_name": "Access Control",
    "domain_number": "8",
    "controls": [
        {
            "control_number": "8.1",
            "control_name": "User Endpoint Devices",
            "control_description": "Ensure that information stored on user devices is protected with security configurations and policies."
        }
    ]
}
expected_output_fail = [
    "**0:8:8.1 - User Endpoint Devices**: Ensure that information stored on user devices is protected with security configurations and policies.",
    "**0:8:8.2 - Remote Access Control**: Ensure secure remote access to information systems and networks."
]
def_output = apply_markdown_gap(domain, 0)
assert not expected_output_fail == def_output

In [ ]:
# | export
def merge_sets(
    setA, # First set to merge 
    setB # Second set to merge
) -> list:  # List of merged sets
    "Merge two sets of controls, combining overlapping subsets"
    merged_set = setA.copy()
    result = []
    for subset in setB:
        if merged_set & subset:
            merged_set |= subset
        else:
            result.append(subset)
    result.insert(0, merged_set)
    return result

In [ ]:
setA = {1, 2, 3}
setB = [{2, 4}, {5}]
merge_sets(setA, setB)

[{1, 2, 3, 4}, {5}]

In [ ]:
#| hide
setA = {1, 2, 3}
setB = [{2, 4}, {5}]
result = merge_sets(setA, setB)
assert result[0] == {1, 2, 3, 4}

In [ ]:
#| export
def create_policy_markdown(doamin): # Single `Domain` object of standard
    "Create markdown for policy processing"
    markdown_output = f"""
    ### Domain: {doamin['domain']}
    #### Controls:
    """
    for control in doamin['Controls']:
        markdown_output += f"""
    - **Control ID**: {control['id']}
      - **Name**: {control['name']}
      - **Description**: {control['description']}
    """
    return markdown_output

In [ ]:
test_obj = {"domain": "Test Domain", "Controls": [{"id": "1", "name": "Control Name", "description": "Description"}]}
create_policy_markdown(test_obj)

'\n    ### Domain: Test Domain\n    #### Controls:\n    \n    - **Control ID**: 1\n      - **Name**: Control Name\n      - **Description**: Description\n    '

In [ ]:
#| hide
test_obj = {"domain": "Test Domain", "Controls": [{"id": "1", "name": "Control Name", "description": "Description"}]}
result = create_policy_markdown(test_obj)
assert "Test Domain" in result
assert "Control Name" in result

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()